In [15]:
question = "What are the total sales by state?"

In [16]:
import pandas as pd
import duckdb as db
from openai import OpenAI
import os

In [17]:
retail_sales = pd.read_csv("../data/Red30 Tech US online retail sales.csv")

In [18]:
tbl_name = "retail_sales"

tbl_description = db.sql("DESCRIBE SELECT * FROM " + tbl_name + ";")
tbl_description

┌──────────────┬─────────────┬─────────┬─────────┬─────────┬─────────┐
│ column_name  │ column_type │  null   │   key   │ default │  extra  │
│   varchar    │   varchar   │ varchar │ varchar │ varchar │ varchar │
├──────────────┼─────────────┼─────────┼─────────┼─────────┼─────────┤
│ OrderNum     │ BIGINT      │ YES     │ NULL    │ NULL    │ NULL    │
│ OrderDate    │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ OrderType    │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ CustomerType │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ CustName     │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ CustState    │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ ProdCategory │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ ProdNumber   │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ ProdName     │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ Quantity     │ BIGINT      │ YES     │ NULL    │ NULL    │ NULL    │
│ Pric

In [19]:
col_attr = tbl_description.df()[["column_name", "column_type"]]
col_attr["column_joint"] = col_attr["column_name"] + " " + col_attr["column_type"]
col_names = (
    str(list(col_attr["column_joint"].values))
    .replace("[", "")
    .replace("]", "")
    .replace("'", "")
)


In [20]:
print(col_names)

OrderNum BIGINT, OrderDate VARCHAR, OrderType VARCHAR, CustomerType VARCHAR, CustName VARCHAR, CustState VARCHAR, ProdCategory VARCHAR, ProdNumber VARCHAR, ProdName VARCHAR, Quantity BIGINT, Price DOUBLE, Discount DOUBLE, OrderTotal DOUBLE


In [21]:
system_template = """
Given the following SQL table, your job is to write queries given a user’s request. \n
CREATE TABLE {} ({}) \n
"""


In [22]:
user_template = "Write a SQL query that returns - {}"


In [23]:
system = system_template.format(tbl_name, col_names)
print(system)



Given the following SQL table, your job is to write queries given a user’s request. 

CREATE TABLE retail_sales (OrderNum BIGINT, OrderDate VARCHAR, OrderType VARCHAR, CustomerType VARCHAR, CustName VARCHAR, CustState VARCHAR, ProdCategory VARCHAR, ProdNumber VARCHAR, ProdName VARCHAR, Quantity BIGINT, Price DOUBLE, Discount DOUBLE, OrderTotal DOUBLE) 




In [24]:
user = user_template.format(question)
print(user)


Write a SQL query that returns - What are the total sales by state?


In [25]:
openai_api_key = os.environ.get("OPENAI_API_KEY")
max_tokens = 5000
client = OpenAI(api_key=openai_api_key)

In [26]:
response = client.chat.completions.create(
    model="gpt-5",
    messages=[{"role": "system", "content": system}, 
              {"role": "user", "content": user}],
    max_completion_tokens=max_tokens,
)


In [27]:
print(response.choices[0].message.content)


SELECT
  CustState,
  SUM(OrderTotal) AS total_sales
FROM retail_sales
GROUP BY CustState
ORDER BY total_sales DESC;


In [28]:
query = """
SELECT
  CustState,
  SUM(OrderTotal) AS total_sales
FROM retail_sales
GROUP BY CustState
ORDER BY total_sales DESC;
"""


In [29]:
db.sql(query)


┌────────────────┬────────────────────┐
│   CustState    │    total_sales     │
│    varchar     │       double       │
├────────────────┼────────────────────┤
│ New York       │  616925.8400000001 │
│ California     │  540285.5199999997 │
│ Florida        │  394483.7399999998 │
│ Texas          │ 349925.48000000004 │
│ North Carolina │  345118.2600000001 │
│ Pennsylvania   │ 290120.09000000014 │
│ Minnesota      │ 267308.95000000007 │
│ Washington     │ 257140.79999999993 │
│ Virginia       │ 248797.89000000004 │
│ Georgia        │ 237607.26000000004 │
│    ·           │          ·         │
│    ·           │          ·         │
│    ·           │          ·         │
│ New Mexico     │ 36696.380000000005 │
│ New Jersey     │            35971.9 │
│ North Dakota   │            29907.5 │
│ West Virginia  │           23967.97 │
│ Nebraska       │           22604.36 │
│ Wyoming        │  9712.550000000001 │
│ Wisconsin      │  7345.029999999999 │
│ South Dakota   │            4071.62 │
